# Homeworks
1. Find the answer to the question raised in the lab1

    Some helpful resources:
- DeepWalk: https://arxiv.org/pdf/1403.6652.pdf
- Word2vec: https://arxiv.org/pdf/1301.3781.pdf
- Repository Github of Word2vec at [this link](https://github.com/RaRe-Technologies/gensim)
2. Implement a simple word2vec algorithm for the DeepWalk (Attributes for each node should be created).
3. Use some libraries to solve a real problem

# Answer

## Implement Word2vec

### Download data and install packages

In [14]:
!pip install karateclub==1.2.0
!pip install umap-learn

In [15]:
!pip install numpy==1.23.5
!pip install scipy==1.13.0

In [16]:
!gdown --id "1RmrHId0d-uY7kJCSgCtNYbwYfp4Oum3c&export=download"
!unrar x -Y "/content/lab3.rar" -d "/content/"

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1RmrHId0d-uY7kJCSgCtNYbwYfp4Oum3c&export=download
To: /content/lab3.rar
100% 1.54M/1.54M [00:00<00:00, 43.8MB/s]

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/lab3.rar

Extracting  /content/lab3_attributes.csv                                   0%  OK 
Extracting  /content/facebook_features.json                               28%  OK 
Extracting  /content/facebook_target.csv                                  59%  OK 
Extracting  /content/lab3_edgelist.txt                                    59%  OK 
Extracting  /content/facebook_edges.csv                                  100%  OK 
All OK


### Packages

In [17]:

# Task 1
import networkx as nx
from joblib import Parallel, delayed
import random
import itertools
import numpy as np
import pandas as pd

# Task 2
import json
import umap
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix
from karateclub.utils.walker import RandomWalker
from gensim.models.word2vec import Word2Vec
import seaborn as sns


### Utils

In [18]:
def partition_num(num, workers):
    if num % workers == 0:
        return [num//workers]*workers
    else:
        return [num//workers]*workers + [num % workers]

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


def get_attributes_of_node(node_paths):
  node_paths_attributes = []
  # Get attribute (word) for each node
  df_attr = pd.read_csv("lab3_attributes.csv").astype(str)
  dict_attr = {}
  for i in range(len(df_attr)):
    dict_attr[df_attr.iloc[i, 0]] = df_attr.iloc[i, 1]
  for path in node_paths:
    for index, node in enumerate(path):
      path[index] = dict_attr[node]
    node_paths_attributes.append(path)
  return node_paths_attributes

def preprocessing(sentences):
    training_data = []
    for sentence in sentences:
        x = [word for word in sentence]
        training_data.append(x)
    return training_data


def prepare_data_for_training(sentences,w2v):
    data = {}
    for sentence in sentences:
        for word in sentence:
            if word not in data:
                data[word] = 1
            else:
                data[word] += 1
    V = len(data)
    data = sorted(list(data.keys()))
    vocab = {}
    for i in range(len(data)):
        vocab[data[i]] = i

    for sentence in sentences:
        for i in range(len(sentence)):
            center_word = [0 for x in range(V)]
            center_word[vocab[sentence[i]]] = 1
            context = [0 for x in range(V)]

            for j in range(i-w2v.window_size,i+w2v.window_size):
                if i!=j and j>=0 and j<len(sentence):
                    context[vocab[sentence[j]]] += 1
            w2v.X_train.append(center_word)
            w2v.y_train.append(context)
    w2v.initialize(V,data)

    return w2v.X_train,w2v.y_train

### TO DO

In [31]:
class word2vec:
    def __init__(self, window_size=5, epochs=100, hidden_size=100, learning_rate=0.001):
        self.window_size = window_size
        self.epochs = epochs
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate
        self.X_train = []
        self.y_train = []
        self.vocab = {}
        self.V = None
        self.words = None
        self.w1 = None
        self.w2 = None

    def initialize(self, V, data):
        self.V = V
        self.words = data
        self.vocab = {word: index for index, word in enumerate(data)}
        self.w1 = np.random.uniform(-0.8, 0.8, (self.V, self.hidden_size))
        self.w2 = np.random.uniform(-0.8, 0.8, (self.hidden_size, self.V))

    def train(self):
        for epoch in range(self.epochs):
            loss = 0
            for i in range(len(self.X_train)):
                # Forward propagation
                x = np.array(self.X_train[i])
                y_true = np.array(self.y_train[i])

                h = np.dot(x, self.w1)
                u = np.dot(h, self.w2)
                y_pred = softmax(u)

                # Calculate loss (cross-entropy)
                loss_example = -np.sum(y_true * np.log(y_pred + 1e-8))
                loss += loss_example

                # Backpropagation
                error = y_pred - y_true
                dw2 = np.outer(h, error)  # Gradient for w2
                dw1 = np.outer(x, np.dot(self.w2, error))  # Gradient for w1

                # Update weights
                self.w1 -= self.learning_rate * dw1
                self.w2 -= self.learning_rate * dw2

            if (epoch + 1) % 10 == 0 or epoch == 0:
                print(f'Epoch {epoch + 1}/{self.epochs}, Loss: {loss:.4f}')

    def predict(self, word, number_of_predictions):
        if word not in self.vocab:
            print(f"Word '{word}' not in vocabulary.")
            return []

        index = self.vocab[word]
        embedding = self.w1[index]  # Embedding vector for the input word
        similarities = {}
        for i in range(self.V):
            if i != index:
                other_embedding = self.w1[i]
                # Compute cosine similarity
                cosine_sim = np.dot(embedding, other_embedding) / (np.linalg.norm(embedding) * np.linalg.norm(other_embedding) + 1e-9)
                similarities[self.words[i]] = cosine_sim

        # Sort by similarity
        sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
        top_predictions = [word for word, _ in sorted_similarities[:number_of_predictions]]
        return top_predictions


### DeepWalk

In [20]:
class RandomWalker:
  def __init__(self, G, num_walks, walk_length):
      """
      :param G: Graph
      :param num_walks: a number of walks
      :param walk_length: Length of a walk. Each walk is considered as a sentence
      """
      self.G = G
      self.num_walks = num_walks
      self.walk_length = walk_length


  def deepwalk_walk(self, start_node):
      """
      :param start_node: Starting node of a walk
      """
      walk = [start_node]
      while len(walk) < self.walk_length:
          cur = walk[-1]
          # Check if having any neighbors at the current node
          cur_nbrs = list(self.G.neighbors(cur))
          if len(cur_nbrs) > 0:
              # Random walk with the probability of 1/d(v^t). d(v^t) is the node degree
              walk.append(random.choice(cur_nbrs))
          else:
              break
      return walk


  def simulate_walks(self, workers=1, verbose=0):
      """
      :param workers: a number of workers running in parallel processing
      :param verbose: progress bar
      """
      G = self.G
      nodes = list(G.nodes())
      results = Parallel(n_jobs=workers, verbose=verbose)(
          delayed(self._simulate_walks)(nodes) for num in
          partition_num(self.num_walks, workers))
      walks = list(itertools.chain(*results))
      return walks


  # INFORMATION EXTRACTOR
  def _simulate_walks(self, nodes):
      walks = []
      # Iterate all walks per vertex
      for _ in range(self.num_walks):
          random.shuffle(nodes)
          # Iterate all nodes in a walk
          for v in nodes:
            walks.append(self.deepwalk_walk(start_node=v))
      return walks

In [21]:
class DeepWalk:
    def __init__(self, graph, walk_length, num_walks, workers=1):

        self.graph = graph
        self.w2v_model = None
        self._embeddings = {}

        self.walker = RandomWalker(graph, num_walks=num_walks, walk_length=walk_length)
        self.walks = self.walker.simulate_walks(workers=workers, verbose=1)
        self.sentences = get_attributes_of_node(self.walks)


    def train(self, window_size=5, epochs=100):
        print("Learning embedding vectors...")
        training_data = preprocessing(self.sentences)
        w2v = word2vec(window_size, epochs)
        prepare_data_for_training(training_data, w2v)
        w2v.train()
        print("Learning embedding vectors done!")
        self.w2v_model = w2v


    def test(self, word):
        print(self.w2v_model.predict(word,3))

### Run graph embedding

In [32]:
G = nx.read_edgelist('lab3_edgelist.txt',create_using=nx.DiGraph(),nodetype=None,data=[('weight',int)])# Read graph
model = DeepWalk(G, walk_length=3, num_walks=10, workers=1)#init model
model.train(window_size=5)# train model

Learning embedding vectors...
Epoch 1/100, Loss: 5984.9095
Epoch 10/100, Loss: 3792.8578
Epoch 20/100, Loss: 3897.2405
Epoch 30/100, Loss: 4500.7118
Epoch 40/100, Loss: 5405.7888
Epoch 50/100, Loss: 6144.6849
Epoch 60/100, Loss: 6634.5734
Epoch 70/100, Loss: 7128.0932
Epoch 80/100, Loss: 7814.6353
Epoch 90/100, Loss: 9690.5579
Epoch 100/100, Loss: 13173.0410
Learning embedding vectors done!


In [30]:
print(model.sentences)
model.test("to")
model.test("this")

[['happy', 'wish', 'learned'], ['am', 'that', 'this'], ['so', 'that', 'you'], ['to', 'happy', 'wish'], ['this', 'so', 'that'], ['wish', 'all', 'all'], ['the', 'all', 'you'], ['!', 'am', 'you'], ['that', 'you', 'to'], ['.', 'so', '!'], ['something', 'learned', '!'], ['I', 'to', '!'], ['you', 'to', 'lab'], ['learned', 'this', 'so'], ['in', 'all', 'am'], ['lab', 'happy', 'wish'], ['.', '.', 'happy'], ['this', 'happy', 'that'], ['I', 'happy', 'wish'], ['lab', '.', 'happy'], ['best', 'best', 'best'], ['all', 'all', 'all'], ['you', 'I', 'this'], ['.', '.', 'happy'], ['!', 'am', 'this'], ['this', 'this', 'I'], ['to', 'you', 'you'], ['so', 'I', 'you'], ['wish', 'all', 'am'], ['learned', 'this', 'to'], ['I', 'to', 'you'], ['best', 'I', 'am'], ['I', 'happy', 'wish'], ['all', 'this', 'so'], ['you', 'to', '!'], ['that', 'this', 'so'], ['am', 'am', 'you'], ['lab', '.', 'happy'], ['happy', 'wish', 'learned'], ['you', 'I', 'this'], ['lab', 'am', 'am'], ['something', 'wish', 'that'], ['.', 'so', 'that

## TO DO: Solve a real problem using some libraries

Goal: When we have a large graph dataset like the Facebook dataset below, we want to classify which company (node) will likely belong to a type of page. If we categorize well, we could apply marketing strategies in a domain on a company that we are surveying.
Therefore, our task is to learn a model which can classify a company using related features.


1. Analyze and visualize the dataset Facebook downloaded in [this website](https://snap.stanford.edu/data/facebook-large-page-page-network.html).
2. Use DeepWalk to embed the graph
3. Train a classifier to do the node classification task using the embedding graph from step 2.

You can do many things with the data. I recommend that you could try many tasks with this data, not only the classification task.

### Read data

In [24]:
edges_path = 'facebook_edges.csv'
targets_path = 'facebook_target.csv'
features_path = 'facebook_features.json'
edges = pd.read_csv('facebook_edges.csv')
targets = pd.read_csv('facebook_target.csv')
with open('facebook_features.json', 'r') as f:
    features = json.load(f)
print(features)

### Visualize datasets

Create a graph. If you want to use smaller graph, please try to create one. It will be lighter when running the code.

### Embedding graph using DeepWalk

Embedding graph using DeepWalk

### Train a classifier

Train a classifer from the embedding graph to the target. Here we use the Random Forest classifier.

# THANK YOU
Please dive more into the codes and papers if you are interested.

Thank you for joining all the labs.